In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import copy
import random
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score

%matplotlib inline

In [2]:
dtrain = pd.read_csv("synthetic_train.csv").to_numpy().astype(np.float32)
dquery = pd.read_csv("synthetic_query.csv").to_numpy().astype(np.float32)
dtest = pd.read_csv("synthetic_test.csv").to_numpy().astype(np.float32)

In [3]:
train_x, train_y = dtrain[:, 0:2], dtrain[:, 2:3]
query_x, query_y = dquery[:, 0:2], dquery[:, 2:3]
test_x, test_y = dtest[:, 0:2], dtest[:, 2:3]

In [4]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
query_x = scaler.transform(query_x)
test_x = scaler.transform(test_x)

In [5]:
Train = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
Query = TensorDataset(torch.from_numpy(query_x), torch.from_numpy(query_y))
Test = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

In [6]:
train_y

array([[0.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

In [7]:
train_loader = DataLoader(Train, batch_size  = 128)
query_loader = DataLoader(Query, batch_size = 128)
test_loader = DataLoader(Test, batch_size = 128)

In [8]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    model.train()
    prediction = []
    label = []

    for batch_idx, (data, target) in enumerate(iterator):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        preds = torch.round(output)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * len(target)
        label.extend(target.tolist())
        prediction.extend(preds.reshape(-1).tolist())

    acc = accuracy_score(label, prediction)
    f1 = f1_score(label, prediction)

    return epoch_loss / len(iterator.dataset), acc, f1

def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    model.eval()
    prediction = []
    label = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(iterator):

            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = torch.max(output, 1)
            preds = torch.round(output)
            loss = criterion(output, target)
            epoch_loss += loss.item() * len(target)
            label.extend(target.tolist())
            prediction.extend(preds.reshape(-1).tolist())
            
    acc = accuracy_score(label, prediction)
    f1 = f1_score(label, prediction)

    return epoch_loss / len(iterator.dataset), acc, f1


In [9]:
model = nn.Sequential(
    nn.Linear(2, 50),
    nn.ReLU(),
    nn.Linear(50, 1),
    nn.Sigmoid(),
)


optimizer = optim.Adam(model.parameters(), lr = 0.005)
criterion = nn.BCELoss()
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = model.to(device)
criterion = criterion.to(device)

best_f1 = -float('inf')
best_model = None
best_acc = -float('inf')

for epoch in range(500):

    train_loss, train_acc, train_f1 = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc, val_f1 = evaluate(model, test_loader, criterion, device)

    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model = copy.deepcopy(model)
        best_acc = valid_acc

    print(f'Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}')
    print(f'Epoch: {epoch} | Val. Loss: {valid_loss:.4f} |  Val. Acc: {valid_acc:.4f} |  Val. F1: {val_f1:.4f}')

print("Best Val. F1: {:.4f}, Best Val. Accuarcy: {:.4f}".format(best_f1, best_acc))

Epoch: 0 | Train Loss: 0.6371 | Train Acc: 0.6786 | Train F1: 0.5335
Epoch: 0 | Val. Loss: 0.5924 |  Val. Acc: 0.8595 |  Val. F1: 0.8588
Epoch: 1 | Train Loss: 0.5370 | Train Acc: 0.8816 | Train F1: 0.8758
Epoch: 1 | Val. Loss: 0.4884 |  Val. Acc: 0.8782 |  Val. F1: 0.8821
Epoch: 2 | Train Loss: 0.4282 | Train Acc: 0.8885 | Train F1: 0.8856
Epoch: 2 | Val. Loss: 0.3854 |  Val. Acc: 0.8831 |  Val. F1: 0.8895
Epoch: 3 | Train Loss: 0.3432 | Train Acc: 0.8889 | Train F1: 0.8878
Epoch: 3 | Val. Loss: 0.3192 |  Val. Acc: 0.8821 |  Val. F1: 0.8893
Epoch: 4 | Train Loss: 0.2931 | Train Acc: 0.8880 | Train F1: 0.8874
Epoch: 4 | Val. Loss: 0.2807 |  Val. Acc: 0.8841 |  Val. F1: 0.8917
Epoch: 5 | Train Loss: 0.2665 | Train Acc: 0.8865 | Train F1: 0.8863
Epoch: 5 | Val. Loss: 0.2599 |  Val. Acc: 0.8890 |  Val. F1: 0.8962
Epoch: 6 | Train Loss: 0.2525 | Train Acc: 0.8860 | Train F1: 0.8858
Epoch: 6 | Val. Loss: 0.2480 |  Val. Acc: 0.8890 |  Val. F1: 0.8964
Epoch: 7 | Train Loss: 0.2450 | Train Acc

Epoch: 61 | Train Loss: 0.2048 | Train Acc: 0.8983 | Train F1: 0.8990
Epoch: 61 | Val. Loss: 0.1985 |  Val. Acc: 0.9057 |  Val. F1: 0.9113
Epoch: 62 | Train Loss: 0.2043 | Train Acc: 0.8988 | Train F1: 0.8995
Epoch: 62 | Val. Loss: 0.1981 |  Val. Acc: 0.9067 |  Val. F1: 0.9121
Epoch: 63 | Train Loss: 0.2038 | Train Acc: 0.8988 | Train F1: 0.8995
Epoch: 63 | Val. Loss: 0.1975 |  Val. Acc: 0.9077 |  Val. F1: 0.9131
Epoch: 64 | Train Loss: 0.2033 | Train Acc: 0.8993 | Train F1: 0.9000
Epoch: 64 | Val. Loss: 0.1971 |  Val. Acc: 0.9077 |  Val. F1: 0.9131
Epoch: 65 | Train Loss: 0.2028 | Train Acc: 0.9002 | Train F1: 0.9009
Epoch: 65 | Val. Loss: 0.1967 |  Val. Acc: 0.9067 |  Val. F1: 0.9121
Epoch: 66 | Train Loss: 0.2022 | Train Acc: 0.9002 | Train F1: 0.9009
Epoch: 66 | Val. Loss: 0.1961 |  Val. Acc: 0.9077 |  Val. F1: 0.9130
Epoch: 67 | Train Loss: 0.2017 | Train Acc: 0.9002 | Train F1: 0.9009
Epoch: 67 | Val. Loss: 0.1956 |  Val. Acc: 0.9077 |  Val. F1: 0.9130
Epoch: 68 | Train Loss: 0.2

Epoch: 120 | Train Loss: 0.1716 | Train Acc: 0.9229 | Train F1: 0.9227
Epoch: 120 | Val. Loss: 0.1707 |  Val. Acc: 0.9224 |  Val. F1: 0.9268
Epoch: 121 | Train Loss: 0.1710 | Train Acc: 0.9229 | Train F1: 0.9227
Epoch: 121 | Val. Loss: 0.1701 |  Val. Acc: 0.9224 |  Val. F1: 0.9268
Epoch: 122 | Train Loss: 0.1704 | Train Acc: 0.9229 | Train F1: 0.9227
Epoch: 122 | Val. Loss: 0.1696 |  Val. Acc: 0.9224 |  Val. F1: 0.9268
Epoch: 123 | Train Loss: 0.1698 | Train Acc: 0.9229 | Train F1: 0.9227
Epoch: 123 | Val. Loss: 0.1692 |  Val. Acc: 0.9214 |  Val. F1: 0.9258
Epoch: 124 | Train Loss: 0.1692 | Train Acc: 0.9243 | Train F1: 0.9242
Epoch: 124 | Val. Loss: 0.1686 |  Val. Acc: 0.9224 |  Val. F1: 0.9268
Epoch: 125 | Train Loss: 0.1686 | Train Acc: 0.9243 | Train F1: 0.9242
Epoch: 125 | Val. Loss: 0.1681 |  Val. Acc: 0.9234 |  Val. F1: 0.9278
Epoch: 126 | Train Loss: 0.1681 | Train Acc: 0.9258 | Train F1: 0.9256
Epoch: 126 | Val. Loss: 0.1677 |  Val. Acc: 0.9234 |  Val. F1: 0.9276
Epoch: 127 | 

Epoch: 179 | Train Loss: 0.1392 | Train Acc: 0.9415 | Train F1: 0.9412
Epoch: 179 | Val. Loss: 0.1383 |  Val. Acc: 0.9430 |  Val. F1: 0.9468
Epoch: 180 | Train Loss: 0.1387 | Train Acc: 0.9425 | Train F1: 0.9422
Epoch: 180 | Val. Loss: 0.1377 |  Val. Acc: 0.9430 |  Val. F1: 0.9468
Epoch: 181 | Train Loss: 0.1382 | Train Acc: 0.9425 | Train F1: 0.9422
Epoch: 181 | Val. Loss: 0.1371 |  Val. Acc: 0.9440 |  Val. F1: 0.9478
Epoch: 182 | Train Loss: 0.1377 | Train Acc: 0.9425 | Train F1: 0.9422
Epoch: 182 | Val. Loss: 0.1366 |  Val. Acc: 0.9440 |  Val. F1: 0.9478
Epoch: 183 | Train Loss: 0.1372 | Train Acc: 0.9435 | Train F1: 0.9433
Epoch: 183 | Val. Loss: 0.1360 |  Val. Acc: 0.9440 |  Val. F1: 0.9478
Epoch: 184 | Train Loss: 0.1367 | Train Acc: 0.9435 | Train F1: 0.9433
Epoch: 184 | Val. Loss: 0.1355 |  Val. Acc: 0.9440 |  Val. F1: 0.9478
Epoch: 185 | Train Loss: 0.1362 | Train Acc: 0.9440 | Train F1: 0.9437
Epoch: 185 | Val. Loss: 0.1349 |  Val. Acc: 0.9450 |  Val. F1: 0.9487
Epoch: 186 | 

Epoch: 238 | Train Loss: 0.1132 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 238 | Val. Loss: 0.1108 |  Val. Acc: 0.9597 |  Val. F1: 0.9630
Epoch: 239 | Train Loss: 0.1129 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 239 | Val. Loss: 0.1106 |  Val. Acc: 0.9597 |  Val. F1: 0.9630
Epoch: 240 | Train Loss: 0.1125 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 240 | Val. Loss: 0.1102 |  Val. Acc: 0.9597 |  Val. F1: 0.9630
Epoch: 241 | Train Loss: 0.1121 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 241 | Val. Loss: 0.1100 |  Val. Acc: 0.9597 |  Val. F1: 0.9630
Epoch: 242 | Train Loss: 0.1117 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 242 | Val. Loss: 0.1096 |  Val. Acc: 0.9617 |  Val. F1: 0.9648
Epoch: 243 | Train Loss: 0.1114 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 243 | Val. Loss: 0.1093 |  Val. Acc: 0.9617 |  Val. F1: 0.9648
Epoch: 244 | Train Loss: 0.1110 | Train Acc: 0.9587 | Train F1: 0.9585
Epoch: 244 | Val. Loss: 0.1090 |  Val. Acc: 0.9617 |  Val. F1: 0.9648
Epoch: 245 | 

Epoch: 298 | Train Loss: 0.0942 | Train Acc: 0.9681 | Train F1: 0.9680
Epoch: 298 | Val. Loss: 0.0950 |  Val. Acc: 0.9676 |  Val. F1: 0.9702
Epoch: 299 | Train Loss: 0.0939 | Train Acc: 0.9686 | Train F1: 0.9685
Epoch: 299 | Val. Loss: 0.0948 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 300 | Train Loss: 0.0936 | Train Acc: 0.9686 | Train F1: 0.9685
Epoch: 300 | Val. Loss: 0.0945 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 301 | Train Loss: 0.0934 | Train Acc: 0.9686 | Train F1: 0.9685
Epoch: 301 | Val. Loss: 0.0943 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 302 | Train Loss: 0.0931 | Train Acc: 0.9690 | Train F1: 0.9690
Epoch: 302 | Val. Loss: 0.0941 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 303 | Train Loss: 0.0929 | Train Acc: 0.9690 | Train F1: 0.9690
Epoch: 303 | Val. Loss: 0.0939 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 304 | Train Loss: 0.0926 | Train Acc: 0.9695 | Train F1: 0.9695
Epoch: 304 | Val. Loss: 0.0936 |  Val. Acc: 0.9686 |  Val. F1: 0.9711
Epoch: 305 | 

Epoch: 357 | Train Loss: 0.0809 | Train Acc: 0.9774 | Train F1: 0.9773
Epoch: 357 | Val. Loss: 0.0835 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 358 | Train Loss: 0.0807 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 358 | Val. Loss: 0.0833 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 359 | Train Loss: 0.0805 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 359 | Val. Loss: 0.0832 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 360 | Train Loss: 0.0803 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 360 | Val. Loss: 0.0830 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 361 | Train Loss: 0.0801 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 361 | Val. Loss: 0.0829 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 362 | Train Loss: 0.0800 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 362 | Val. Loss: 0.0827 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 363 | Train Loss: 0.0798 | Train Acc: 0.9779 | Train F1: 0.9778
Epoch: 363 | Val. Loss: 0.0826 |  Val. Acc: 0.9725 |  Val. F1: 0.9747
Epoch: 364 | 

Epoch: 416 | Train Loss: 0.0713 | Train Acc: 0.9808 | Train F1: 0.9808
Epoch: 416 | Val. Loss: 0.0750 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 417 | Train Loss: 0.0712 | Train Acc: 0.9808 | Train F1: 0.9808
Epoch: 417 | Val. Loss: 0.0749 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 418 | Train Loss: 0.0710 | Train Acc: 0.9808 | Train F1: 0.9808
Epoch: 418 | Val. Loss: 0.0748 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 419 | Train Loss: 0.0709 | Train Acc: 0.9813 | Train F1: 0.9813
Epoch: 419 | Val. Loss: 0.0746 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 420 | Train Loss: 0.0708 | Train Acc: 0.9813 | Train F1: 0.9813
Epoch: 420 | Val. Loss: 0.0745 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 421 | Train Loss: 0.0706 | Train Acc: 0.9813 | Train F1: 0.9813
Epoch: 421 | Val. Loss: 0.0744 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 422 | Train Loss: 0.0705 | Train Acc: 0.9818 | Train F1: 0.9818
Epoch: 422 | Val. Loss: 0.0743 |  Val. Acc: 0.9745 |  Val. F1: 0.9765
Epoch: 423 | 

Epoch: 475 | Train Loss: 0.0641 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 475 | Val. Loss: 0.0683 |  Val. Acc: 0.9784 |  Val. F1: 0.9801
Epoch: 476 | Train Loss: 0.0640 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 476 | Val. Loss: 0.0682 |  Val. Acc: 0.9784 |  Val. F1: 0.9801
Epoch: 477 | Train Loss: 0.0639 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 477 | Val. Loss: 0.0681 |  Val. Acc: 0.9794 |  Val. F1: 0.9811
Epoch: 478 | Train Loss: 0.0638 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 478 | Val. Loss: 0.0680 |  Val. Acc: 0.9794 |  Val. F1: 0.9811
Epoch: 479 | Train Loss: 0.0637 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 479 | Val. Loss: 0.0679 |  Val. Acc: 0.9794 |  Val. F1: 0.9811
Epoch: 480 | Train Loss: 0.0636 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 480 | Val. Loss: 0.0678 |  Val. Acc: 0.9794 |  Val. F1: 0.9811
Epoch: 481 | Train Loss: 0.0635 | Train Acc: 0.9867 | Train F1: 0.9867
Epoch: 481 | Val. Loss: 0.0677 |  Val. Acc: 0.9794 |  Val. F1: 0.9811
Epoch: 482 | 

In [12]:
torch.save(model, "synthetic.pt")

In [13]:
XX_query = torch.from_numpy(query_x)
XX_val = torch.from_numpy(val_x)
XX_test = torch.from_numpy(test_x)

NameError: name 'val_x' is not defined

In [ ]:
best_model.eval()

with torch.no_grad():
    yy_query_p = torch.round(model(XX_query)).numpy()
    yy_val_p = torch.round(model(XX_val)).numpy()
    yy_test_p = torch.round(model(XX_test)).numpy()


In [ ]:
dquery[:, 0:2]

In [ ]:
ddquery = np.concatenate((dquery[:, 0:2], yy_query_p), axis = 1)
ddval = np.concatenate((dval[:, 0:2], yy_val_p), axis = 1)
ddtest = np.concatenate((dtest[:, 0:2], yy_test_p), axis = 1)

In [ ]:
np.save("synthetic_query_v1.npy", ddquery)
np.save("synthetic_val_v1.npy", ddval)
np.save("synthetic_test_v1.npy", ddtest)

In [ ]:
print(yy_query_p)

In [ ]:
print(dquery[:, 0:2])

In [ ]:
print(len(dquery))